In [0]:
dbutils.fs.ls("/mnt/fake/")
spark.conf.set('spark.sql.shuffle.partitions',64)

In [0]:
df = spark.read.parquet("/mnt/fake/generated")

In [0]:
print(f"Number of documents: {df.count():,}")

Number of documents: 1,000,000

In [0]:
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import when
df = df.withColumn('exited',df.exited.cast(DoubleType()))

In [0]:
from pyspark.ml.feature import StringIndexer
GeoIndexer = StringIndexer(inputCol="geography", outputCol="ix_geo").fit(df)
GenderIndexer = StringIndexer(inputCol="gender", outputCol="ix_gender").fit(df)

In [0]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(
  inputCols = ["credit_score","balance","estimated_salary","num_of_products", "ix_geo","ix_gender"],
  outputCol = "features")

In [0]:
from mmlspark.lightgbm import LightGBMClassifier
classifier = LightGBMClassifier(
  featuresCol="features",
  labelCol="exited",
  learningRate=0.3,
  numIterations=50,
  numLeaves=20)

In [0]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[GeoIndexer,GenderIndexer,assembler, classifier])
model = pipeline.fit(df)